In [1]:
import sys, os
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras import utils

In [2]:
#load data and look at the df.

df=pd.read_csv('fer2013.csv')

print(df.head())

   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training


In [3]:
#build the training and test sets 
X_train,train_y,X_test,test_y=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

In [4]:
#set hyperparameters and preprocess data.

num_features = 64
num_labels = 7
batch_size = 64
epochs = 200
width, height = 48, 48

X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

train_y=utils.to_categorical(train_y, num_classes=num_labels)
test_y=utils.to_categorical(test_y, num_classes=num_labels)

In [5]:
#normalizing data between 0 and 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

print(f"shape:{X_train.shape}")

shape:(28709, 48, 48, 1)


In [6]:
##designing the cnn
#1st convolution layer
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#2nd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#3rd convolution layer
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 9, 128)         0

In [7]:
#Compliling the model
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

In [8]:
model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)

Epoch 1/200
449/449 [==============================] - 199s 441ms/step - loss: 1.8051 - accuracy: 0.2536 - val_loss: 1.5691 - val_accuracy: 0.3678
Epoch 2/200
449/449 [==============================] - 195s 434ms/step - loss: 1.5574 - accuracy: 0.3815 - val_loss: 1.4222 - val_accuracy: 0.4494
Epoch 3/200
449/449 [==============================] - 194s 432ms/step - loss: 1.4196 - accuracy: 0.4494 - val_loss: 1.3117 - val_accuracy: 0.4987
Epoch 4/200
449/449 [==============================] - 196s 436ms/step - loss: 1.3341 - accuracy: 0.4814 - val_loss: 1.2975 - val_accuracy: 0.4993
Epoch 5/200
449/449 [==============================] - 194s 432ms/step - loss: 1.2807 - accuracy: 0.5019 - val_loss: 1.2502 - val_accuracy: 0.5132
Epoch 6/200
449/449 [==============================] - 205s 456ms/step - loss: 1.2410 - accuracy: 0.5243 - val_loss: 1.2127 - val_accuracy: 0.5481
Epoch 7/200
449/449 [==============================] - 204s 454ms/step - loss: 1.1953 - accuracy: 0.5470 - val_loss: 1

449/449 [==============================] - 190s 424ms/step - loss: 0.4781 - accuracy: 0.8289 - val_loss: 1.6112 - val_accuracy: 0.5762
Epoch 57/200
449/449 [==============================] - 192s 428ms/step - loss: 0.4667 - accuracy: 0.8314 - val_loss: 1.5509 - val_accuracy: 0.5756
Epoch 58/200
449/449 [==============================] - 191s 425ms/step - loss: 0.4758 - accuracy: 0.8289 - val_loss: 1.6487 - val_accuracy: 0.5762
Epoch 59/200
449/449 [==============================] - 192s 428ms/step - loss: 0.4466 - accuracy: 0.8386 - val_loss: 1.5875 - val_accuracy: 0.5648
Epoch 60/200
449/449 [==============================] - 193s 430ms/step - loss: 0.4503 - accuracy: 0.8362 - val_loss: 1.6135 - val_accuracy: 0.5840
Epoch 61/200
449/449 [==============================] - 183s 408ms/step - loss: 0.4437 - accuracy: 0.8424 - val_loss: 1.6156 - val_accuracy: 0.5798
Epoch 62/200
449/449 [==============================] - 186s 415ms/step - loss: 0.4301 - accuracy: 0.8455 - val_loss: 1.5974 

449/449 [==============================] - 250s 557ms/step - loss: 0.3192 - accuracy: 0.8925 - val_loss: 1.8958 - val_accuracy: 0.5717
Epoch 112/200
449/449 [==============================] - 226s 503ms/step - loss: 0.2939 - accuracy: 0.9013 - val_loss: 1.9032 - val_accuracy: 0.5712
Epoch 113/200
449/449 [==============================] - 182s 405ms/step - loss: 0.3019 - accuracy: 0.8967 - val_loss: 1.9157 - val_accuracy: 0.5673
Epoch 114/200
449/449 [==============================] - 182s 405ms/step - loss: 0.3077 - accuracy: 0.8991 - val_loss: 1.8880 - val_accuracy: 0.5723
Epoch 115/200
449/449 [==============================] - 181s 402ms/step - loss: 0.2961 - accuracy: 0.8991 - val_loss: 1.8203 - val_accuracy: 0.5779
Epoch 116/200
449/449 [==============================] - 180s 402ms/step - loss: 0.3090 - accuracy: 0.8961 - val_loss: 1.8391 - val_accuracy: 0.5860
Epoch 117/200
449/449 [==============================] - 181s 402ms/step - loss: 0.3010 - accuracy: 0.8974 - val_loss: 1

449/449 [==============================] - 182s 405ms/step - loss: 0.2644 - accuracy: 0.9177 - val_loss: 2.1031 - val_accuracy: 0.5768
Epoch 167/200
449/449 [==============================] - 182s 405ms/step - loss: 0.2593 - accuracy: 0.9209 - val_loss: 2.0555 - val_accuracy: 0.5726
Epoch 168/200
449/449 [==============================] - 182s 404ms/step - loss: 0.2560 - accuracy: 0.9195 - val_loss: 2.1986 - val_accuracy: 0.5784
Epoch 169/200
449/449 [==============================] - 182s 405ms/step - loss: 0.2557 - accuracy: 0.9206 - val_loss: 1.9873 - val_accuracy: 0.5756
Epoch 170/200
449/449 [==============================] - 182s 405ms/step - loss: 0.2629 - accuracy: 0.9171 - val_loss: 2.0624 - val_accuracy: 0.5762
Epoch 171/200
449/449 [==============================] - 183s 408ms/step - loss: 0.2475 - accuracy: 0.9187 - val_loss: 1.9313 - val_accuracy: 0.5751
Epoch 172/200
449/449 [==============================] - 182s 405ms/step - loss: 0.2705 - accuracy: 0.9122 - val_loss: 2

In [9]:
#Saving the  model to  use it later on
fer_json2 = model.to_json()
with open("fer2.json", "w") as json_file:
    json_file.write(fer_json2)
model.save_weights("fer2.h5")